# Insiders Clustering Project

# 0.0 General Area

## 0.1 Imports

In [3]:
import pandas  as pd
import numpy   as np
import seaborn as sns
import inflection
import warnings
import re

from matplotlib import pyplot        as plt
from sklearn    import cluster       as c
from sklearn    import metrics       as m
from sklearn    import preprocessing as pp
from sklearn    import decomposition as dd
from sklearn    import ensemble      as en
from plotly     import express       as px

from sklearn.manifold    import TSNE
#from ydata_profiling     import ProfileReport
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

import umap.umap_ as umap

## 0.2 Support Functions

In [4]:
# data description functions
def numerical_attributes( df ):
    df_num_attributes = df.select_dtypes( include=['int64', 'float64'] )

    # Central Tendency - mean, median
    df_mean   = pd.DataFrame( df_num_attributes.apply( np.mean ) ).T
    df_median = pd.DataFrame( df_num_attributes.apply( np.median ) ).T
    
    # Dispersion - min, max, range, std, skew, kurtosis
    df_min =      pd.DataFrame( df_num_attributes.apply( np.min ) ).T
    df_max =      pd.DataFrame( df_num_attributes.apply( np.max ) ).T
    df_range =    pd.DataFrame( df_num_attributes.apply( lambda x: x.max() - x.min() ) ).T
    df_std =      pd.DataFrame( df_num_attributes.apply( np.std ) ).T
    df_skew =     pd.DataFrame( df_num_attributes.apply( lambda x: x.skew() ) ).T
    df_kurtosis = pd.DataFrame( df_num_attributes.apply( lambda x: x.kurtosis() ) ).T
    
    # concatenate dataframes
    df_numerical = pd.concat( [df_min, df_max, df_range, df_mean, df_median, df_std, df_skew, df_kurtosis] ).T.reset_index()
    df_numerical.columns =  ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis'] 
    df_numerical
    
    return df_numerical

def categorical_attributes( df ):
    df_categorical = df.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'] )

    return df_categorical

# data cleaning functions
def rename_columns( df ):
    title = lambda x: inflection.titleize( x )
    snakecase = lambda x: inflection.underscore( x )
    spaces = lambda x: x.replace(" ", "")

    cols_old = list( df.columns )
    cols_old = list( map( title, cols_old ) )
    cols_old = list( map( spaces, cols_old ) )
    cols_new = list( map( snakecase, cols_old ) )
    
    df.columns = cols_new

    df = df.rename( columns = {'customer' : 'customer_id' } )

    return df

# feature engineering functions
def feature_creation( df ):
    # creating gross revenue
    df['gross_revenue'] = df['quantity'] * df['unit_price']    
    
    return df

def jupyter_settings():
    %matplotlib inline
    #%pylab inline

    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24

    #display( HTML( '<style>.container { width:100% !important; }</style>') )

    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )

    sns.set()

jupyter_settings()
warnings.filterwarnings( 'ignore' )

## 0.3 Load Data

In [ ]:
df_raw = pd.read_csv( '../data/Ecommerce.csv', low_memory=False, encoding='latin1')
df_raw.head()

# 1.0 Data Description

In [ ]:
df1 = df_raw.copy()

## 1.1 Data Dimension

In [ ]:
print(f'Number of rows: { df1.shape[0] } ')
print(f'Number of cols: { df1.shape[1] } ')

## 1.2 List Data Types

In [ ]:
df1.dtypes

## 1.3 Descriptive Statistics

In [ ]:
# identify and separate numerical and categorical attrributes
df_numerical   = numerical_attributes( df1 )
df_categorical = categorical_attributes( df1 )

### 1.3.1 Numerical Attributes

In [ ]:
df_numerical.head()

### 1.3.2 Categorical Attributes

In [ ]:
df_categorical.head()

# 2.0 Data Cleaning

In [ ]:
df2 = df1.copy()

## 2.1 Rename Columns

In [ ]:
df2 = rename_columns( df2 )
df2.head()

## 2.2 Drop Columns

In [ ]:
df2 = df2.drop( 'unnamed:8', axis=1 )
df2.head()

## 2.3 Check N/A

In [ ]:
# number of rows with NA values by column
df1.isna().sum()

## 2.4 Fillout N/A

### customer_id

In [ ]:
#df2['customer_id'].max() = 1828

df_missing = df2[df2['customer_id'].isna()]
df_missing = pd.DataFrame( df_missing['invoice_no'].drop_duplicates() )
df_missing['customer_id'] = np.arange( 1900, 1900+len( df_missing ), 1 )

df_missing.head()

In [ ]:
# merging with missing customer_id dataframe
df2 = pd.merge( df2, df_missing, how='left', on='invoice_no' )
df2.head()

In [ ]:
# apply coalesce
df2['customer_id'] = df2['customer_id_x'].combine_first( df2['customer_id_y'] )

# drop extra columns
df2 =df2.drop( columns=['customer_id_x', 'customer_id_y'], axis=1 )

df2.head()

In [ ]:
# re-check n/a values
df2.isna().sum()

## 2.5 Drop N/A Values

In [ ]:
df2 = df2.dropna( subset = ['description', 'customer_id'] )

print(f'Removed data : { round( 1 - ( df2.shape[0] / df_raw.shape[0] ), 2) } ') 
print(f'Number of rows Updated: { df2.shape[0] } ')

## 2.6 Change Data Types

In [ ]:
df2['invoice_date'] = pd.to_datetime( df2['invoice_date'] )
df2.head()

## 2.7 Remove Rows

### invoice_no

In [ ]:
# identify invoice_no with letters
index = df2['invoice_no'].apply( lambda x : bool( re.search( '[^0-9]+', x ) ) )
df_letter_invoices =  df2.loc[ index, :] 
df_letter_invoices.head()

### stock_code

In [ ]:
# identify stock_code with only letters
index = df2['stock_code'].apply( lambda x : bool( re.search( '^[a-zA-Z]+$', x ) ) )
df_letter_stock_code = df2.loc[ index, :] 

list_stock_code = df_letter_stock_code['stock_code'].unique()
print(f'Rows removed: { len( df2[ df2["stock_code"].isin( list_stock_code ) ] ) } ' )

df2 = df2[ ~df2['stock_code'].isin( list_stock_code ) ]

In [ ]:
list_stock_code = df_letter_stock_code['stock_code'].unique()
print(f'Rows removed: { len( df2[ df2["stock_code"].isin( list_stock_code ) ] ) } ' )

df2 = df2[ ~df2['stock_code'].isin( list_stock_code ) ]

### unit_price

In [ ]:
# removing rows when unit_price < 0.04
print(f'Rows removed: { len( df2[ df2["unit_price"] < 0.04] ) } ' )
df2 = df2[ df2['unit_price'] >= 0.04]

### country

In [ ]:
df2['country'].unique()

In [ ]:
# removing rows where are no contries
print(f'Rows removed: { len( df2[ df2["country"].isin( ["European Community", "Unspecified"] ) ] ) } ' )
df2 = df2[ ~df2['country'].isin( ['European Community', 'Unspecified'] ) ]

## 2.8 Separate original dataset in two: purchases and returns

In [ ]:
df_returns   = df2[ df2['quantity'] < 0]
df_purchases = df2[ df2['quantity'] > 0]

# 3.0 Feature Engineering

In [ ]:
df3 = df_purchases.copy()

## 3.1 Feature Creation

In [ ]:
def feature_creation( df, df_returns ):
    # creating gross revenue
    df['gross_revenue'] = df['quantity'] * df['unit_price']    

    # creating customer dataframe
    df_customer = df.drop( columns= ['invoice_no', 'stock_code', 'description', 'quantity', 
                                      'invoice_date', 'unit_price', 'country', 'gross_revenue'], axis=1).drop_duplicates( ignore_index=True )

    # creating customer monetary dataframe - amount of purchases
    df_monetary = df[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()

    # creating customer frequency dataframe - number of purchases
    df_frequency = df[['customer_id', 'invoice_no']].groupby('customer_id').nunique().reset_index()

    # creating customer recency dataframe - number of days since last purchase
    max_invoice_date = df['invoice_date'].max()

    df_recency = df[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
    df_recency['recency_days'] =  ( max_invoice_date - df_recency['invoice_date'] ).dt.days
    df_recency = df_recency.drop( 'invoice_date', axis=1 )

    # creating average customer ticket dataframe
    df_avg_ticket = ( df[['customer_id', 'gross_revenue']]
                        .groupby('customer_id')
                        .mean()
                        .reset_index()
                        .rename( columns={ 'gross_revenue' : 'avg_ticket' } ) )

    # creating quantity products purchased dataframe
    df_prod_purchased = ( df[['customer_id', 'quantity']]
                         .groupby('customer_id')
                         .count()
                         .reset_index()
                         .rename( columns= {'quantity' : 'qtt_prod_purchased' } ) )

    # creating purchase frequency dataframe = qtt_purchases / range_days
    df_aux = ( df[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                                .groupby( 'customer_id' )
                                                                .agg( max_ =  ( 'invoice_date', 'max' ),
                                                                      min_ =  ( 'invoice_date', 'min' ),
                                                                      days_ = ( 'invoice_date', lambda x : ( x.max() - x.min() ).days + 1),
                                                                      buys_ = ( 'invoice_no', 'count' ) ) ).reset_index()
    # calculate purchase frequency 
    df_aux['purchase_frequency'] = df_aux[['buys_', 'days_']].apply( lambda x : x['buys_'] / x['days_'] if x['days_'] != 0 else 0, axis=1 )
    df_purc_freq = df_aux[['customer_id', 'purchase_frequency']]

    # number of returns
    df_returns = df_returns[['customer_id', 'quantity']].groupby( 'customer_id' ).sum().reset_index().rename( columns={ 'quantity' : 'qtt_returns' } )
    df_returns['qtt_returns'] = abs( df_returns['qtt_returns'] )

    # basket size - quantity
    df_bsize_quantity = ( df_purchases[['customer_id', 'invoice_no', 'quantity']]
                                .groupby('customer_id')
                                .agg( n_purchases=( 'invoice_no', 'nunique' ),
                                      n_products=( 'quantity', 'sum' ) )
                                .reset_index() )

    df_bsize_quantity['avg_bsize_quantity'] = df_bsize_quantity['n_products'] / df_bsize_quantity['n_purchases'] 
    df_bsize_quantity = df_bsize_quantity.drop( columns=['n_products', 'n_purchases'], axis=1 )

    # basket size - variety
    df_bsize_variety = ( df_purchases[['customer_id', 'invoice_no', 'stock_code']]
                                    .groupby('customer_id')
                                    .agg( n_purchases=( 'invoice_no', 'nunique' ),
                                          n_products=( 'stock_code', 'nunique' ) )
                                    .reset_index() )
    
    df_bsize_variety['avg_bsize_variety'] = df_bsize_variety['n_products'] / df_bsize_variety['n_purchases'] 
    df_bsize_variety = df_bsize_variety.drop( columns=['n_products', 'n_purchases'], axis=1)

    # merging customer dataframes
    df_customer = pd.merge( df_customer, df_recency,         how='left', on='customer_id' )
    df_customer = pd.merge( df_customer, df_frequency,       how='left', on='customer_id' )
    df_customer = pd.merge( df_customer, df_monetary,        how='left', on='customer_id' )
    df_customer = pd.merge( df_customer, df_avg_ticket,      how='left', on='customer_id' )
    df_customer = pd.merge( df_customer, df_prod_purchased,  how='left', on='customer_id' )
    df_customer = pd.merge( df_customer, df_purc_freq,       how='left', on='customer_id' )
    df_customer = pd.merge( df_customer, df_returns,         how='left', on='customer_id' )
    df_customer = pd.merge( df_customer, df_bsize_quantity,  how='left', on='customer_id' )
    df_customer = pd.merge( df_customer, df_bsize_variety,   how='left', on='customer_id' )

    df_customer.loc[ df_customer['qtt_returns'].isna(), 'qtt_returns'] = 0

    return df_customer
    

In [ ]:
df3 = feature_creation( df3, df_returns )
df3.head()

In [ ]:
df3.isna().sum()

In [ ]:
# average recency days
df2 = df_purchases.copy()
df_aux = df2[['customer_id', 'invoice_date']].drop_duplicates().sort_values( ['customer_id', 'invoice_date'], ascending=[False, False] )
df_aux['next_customer_id']  = df_aux['customer_id'].shift()
df_aux['next_invoice_date'] = df_aux['invoice_date'].shift()
df_aux['avg_recency_days']  = df_aux.apply( lambda x : ( x['next_invoice_date'] - x['invoice_date'] ).days if x['customer_id'] == x['next_customer_id'] 
                                            else np.nan, axis=1 )

df_aux.head(10)


# 4.0 Exploratory Data Analysis

In [ ]:
df4 = df3.copy()

## 4.1 Univariate Analysis

In [ ]:
profile = ProfileReport(df4, title="EDA - Insiders Clustering")
profile.to_file('eda_insiders_clustering_v_01.html')

**Notes**
1. Frequency has low variance
2. Avg Ticket has low variance

## 4.2 Bivariate Analysis

In [ ]:
df4_aux = df4.drop( ['customer_id'], axis=1 )

#plt.figure( figsize=(25,12) )
sns.pairplot( df4_aux )

#df_viz = df9.drop( 'customer_id', axis=1 )
#sns.pairplot( df_viz, hue='cluster');

In [ ]:
df4_aux.head()

## 4.3 Study of Space

In [ ]:
df_space = df4.drop( ['customer_id'], axis=1 )

mm = pp.MinMaxScaler()

df_space['recency_days']       = mm.fit_transform( df_space[['recency_days']] )
df_space['invoice_no']         = mm.fit_transform( df_space[['invoice_no']] ) 
df_space['gross_revenue']      = mm.fit_transform( df_space[['gross_revenue']] )
df_space['avg_ticket']         = mm.fit_transform( df_space[['avg_ticket']] )
df_space['qtt_prod_purchased'] = mm.fit_transform( df_space[['qtt_prod_purchased']] )
df_space['purchase_frequency'] = mm.fit_transform( df_space[['purchase_frequency']] )
df_space['qtt_returns']        = mm.fit_transform( df_space[['qtt_returns']] )
df_space['avg_bsize_quantity'] = mm.fit_transform( df_space[['avg_bsize_quantity']] )
df_space['avg_bsize_variety']  = mm.fit_transform( df_space[['avg_bsize_variety']] )

df_space.head()

In [ ]:
df_space.head()

### 4.3.1 PCA

In [ ]:
# PCA defined by numers of features
pca = dd.PCA( n_components=df_space.shape[1] )
principal_components = pca.fit_transform( df_space )

# plot explained variable
features = range( pca.n_components_ )
plt.bar( features, pca.explained_variance_ratio_, color='black' );


In [ ]:
# creating dataframe with pca components
df_pca = pd.DataFrame( principal_components )

sns.scatterplot( x=0, y=1, data=df_pca );

### 4.3.2 UMAP

In [ ]:
reducer = umap.UMAP( random_state=42 )
embedding = reducer.fit_transform( df_space )

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

# plot UMAP
sns.scatterplot( x='embedding_x', 
                 y='embedding_y',
                 data=df_pca );

### 4.3.3 t-SNE

In [ ]:
reducer =    TSNE( n_components=2, random_state=42, n_jobs=-1 )
embedding = reducer.fit_transform( df_space )

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

# plot UMAP
sns.scatterplot( x='embedding_x', 
                 y='embedding_y',
                 data=df_pca );

### 4.4.4 Tree-Based Embedding

In [ ]:
# creating X and y values for training
X = df_space.drop( ['gross_revenue'], axis=1 )
y = df_space['gross_revenue']

# model definition
rf_model = en.RandomForestRegressor( n_estimators=100, random_state=42 )

# model training
rf_model.fit( X, y )

In [ ]:
df_leaf = pd.DataFrame( rf_model.apply( X ) )
df_leaf.head()

In [ ]:
df_leaf.shape

In [ ]:
reducer = umap.UMAP( random_state=42 )
embedding = reducer.fit_transform( df_leaf )

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

# plot UMAP
sns.scatterplot( x='embedding_x', 
                 y='embedding_y',
                 data=df_pca );

# 5.0 Data Preparation

In [ ]:
df5 = df4.copy()

In [ ]:
df5.columns

## 5.1 Standard Scaler

In [ ]:
ss = pp.StandardScaler()

df5['gross_revenue']      = ss.fit_transform( df5[['gross_revenue']] )
df5['recency_days']       = ss.fit_transform( df5[['recency_days']] )
df5['invoice_no']         = ss.fit_transform( df5[['invoice_no']] )
df5['avg_ticket']         = ss.fit_transform( df5[['avg_ticket']] )
df5['qtt_prod_purchased'] = ss.fit_transform( df5[['qtt_prod_purchased']] )
df5['purchase_frequency'] = ss.fit_transform( df5[['purchase_frequency']] )
df5['qtt_returns']        = ss.fit_transform( df5[['qtt_returns']] )
df5['avg_bsize_quantity'] = ss.fit_transform( df5[['avg_bsize_quantity']] )
df5['avg_bsize_variety']  = ss.fit_transform( df5[['avg_bsize_variety']] )

# 6.0 Feature Selection

In [ ]:
df6 = df5.copy()

# 7.0 Hyperparameter Fine-Tuning

In [ ]:
df7 = df6.copy()

X = df7.drop( columns='customer_id' )
X.head()


## 7.1 Within-Cluster Sum of Square (WSS)

In [ ]:
clusters = [2, 3, 4, 5, 6, 7]
kmeans = KElbowVisualizer( c.KMeans(), k=clusters, timing=False )
kmeans.fit( X )
kmeans.show();

## 7.2 Silhouette Score

In [ ]:
clusters = [2, 3, 4, 5, 6, 7]
kmeans = KElbowVisualizer( c.KMeans(), k=clusters, metric='silhouette', timing=False )
kmeans.fit( X )
kmeans.show();

## 7.3 Silhouette Analysis

In [ ]:
clusters = [2, 3, 4, 5, 6, 7]
fig, axes = plt.subplots(3, 2, figsize=(25, 18) )

for k in clusters:
    q, mod = divmod(k, 2)
    
    kmeans = c.KMeans( init='random', n_clusters=k, n_init=10, max_iter=100, random_state=42 )
    visualizer = SilhouetteVisualizer( kmeans, colors='yellowbrick', ax=axes[q-1][mod])
    visualizer.fit( X )
    visualizer.finalize()

# 8.0 Model Training

In [ ]:
df8 = df7.copy()

## 8.1 K-Means

In [ ]:
# model definition
k=5
kmeans = c.KMeans( init='random', n_clusters=k, n_init=10, max_iter=300, random_state=42 )

# model traning
kmeans.fit( X )

# clustering
labels = kmeans.labels_


## 8.2 Cluster Validation

In [ ]:
# WSS - Within Cluster Sum of Square
print(f'WSS : { kmeans.inertia_ } ' )

## SS - Silhouette Score
print(f'SS : { m.silhouette_score( X, labels, metric="euclidean" ) } ' )

# 9.0 Cluster Analysis

In [ ]:
df9 = df8.copy()

df9['cluster'] = labels
df9.head()

## 9.1 Visualization Inspection

In [ ]:
#fig = px.scatter_3d( df10, x='recency_days', y='purchase_frequency', z='gross_revenue', color='cluster' )
#fig.show()

visualizer = SilhouetteVisualizer( kmeans, colors = 'yellowbrick' )
visualizer.fit( X )
visualizer.finalize()

## 9.2 2D Plot

In [ ]:
df_viz = df9.drop( 'customer_id', axis=1 )
sns.pairplot( df_viz, hue='cluster');

## 9.3 UMAP

In [ ]:
# pip install umap-learn

reducer = umap.UMAP( n_neighbors=50, random_state=42 )
embedding = reducer.fit_transform( X )

# embedding
df_viz['embedding_x'] = embedding[:,0]
df_viz['embedding_y'] = embedding[:,1]

# plot UMAP
sns.scatterplot( x='embedding_x', 
                 y='embedding_y',
                 hue='cluster',
                 palette=sns.color_palette( 'hls', n_colors=len( df_viz['cluster'].unique() ) ),
                 data=df_viz );

## 9.4 Cluster Profile

In [ ]:
# Number of customer
df_cluster = df9[['cluster', 'customer_id']].groupby( 'cluster').count().reset_index().rename( columns = { 'customer_id' : 'count' } )
df_cluster['perc_customer'] = round( df_cluster['count'] / df_cluster['count'].sum() * 100, 2)

# Average Recency Days
df_cluster_recency = df9[['cluster', 'recency_days']].groupby( 'cluster').mean().reset_index()
df_cluster_recency['recency_days'] = round( df_cluster_recency['recency_days'], 2)

# Average Purchase Frequeccy
df_cluster_frequency = df9[['cluster', 'invoice_no']].groupby( 'cluster').mean().reset_index()
df_cluster_frequency['invoice_no'] = round( df_cluster_frequency['invoice_no'], 2)

# Average Gross Revenue
df_cluster_monetary = df9[['cluster', 'gross_revenue']].groupby( 'cluster').mean().reset_index()
df_cluster_monetary['gross_revenue'] = round( df_cluster_monetary['gross_revenue'], 2)

# Average Ticket
df_cluster_ticket = df9[['cluster', 'avg_ticket']].groupby( 'cluster').mean().reset_index()
df_cluster_ticket['avg_ticket'] = round( df_cluster_ticket['avg_ticket'], 2)

# Merging datasets
df_cluster = pd.merge( df_cluster, df_cluster_recency,   how='inner', on='cluster' )
df_cluster = pd.merge( df_cluster, df_cluster_frequency, how='inner', on='cluster' )
df_cluster = pd.merge( df_cluster, df_cluster_monetary,  how='inner', on='cluster' )
df_cluster = pd.merge( df_cluster, df_cluster_ticket,    how='inner', on='cluster' )
df_cluster.head()
